In [1]:
import cv2
import pickle
import numpy as np
from deepface import DeepFace
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances
import mlflow
from collections import deque
import os
from datetime import datetime
# import mediapipe.solu

In [2]:
# loading embeddings
with open("face_embeddings.pkl", "rb") as f:
    embeddings = pickle.load(f)

In [3]:
cap =cv2.VideoCapture(0)

In [4]:
THRESHOLD  = 0.7
while True:
    ret,frame = cap.read()
    if not ret:
        break
    try:
        reps = DeepFace.represent(frame,model_name="ArcFace",enforce_detection=False)
        for rep in reps:
            emb = rep['embedding']
            name = 'Unknown'
            min_dist = 1
            for person ,embs in embeddings.items():
                for known_emb in embs:
                    dist = cosine(emb,known_emb)
                    if dist < min_dist :
                        min_dist =dist
                        name = person
            if min_dist < THRESHOLD:
                cv2.putText(frame, name, (50,50),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
            else:
                cv2.putText(frame, "Unknown", (50,50),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

    except:
        pass
    cv2.imshow("face_recognition : ",frame )
    if(cv2.waitKey(1) & 0xFF ==ord("q")):
        break
cap.release()
cv2.destroyAllWindows()

In [10]:

THRESHOLD = 0.5  #setting threshold for judgement it is known person or unknown
pred_history = deque(maxlen=10) # for finding highest probable ans
display_name = "Detecting..."

cap = cv2.VideoCapture(0)  

if not cap.isOpened():
    print("❌ Camera not opened")
    exit()


while True:
    ret, frame = cap.read()
    if not ret:
        break

    try:
        reps = DeepFace.represent(# for convert input to embeddings
            frame,
            model_name="ArcFace",
            enforce_detection=False,
             detector_backend="yolov8n"  
            
        )

        best_person = "Unknown"
        best_dist = 1.0
        second_best = 1.0   

        # comparing cosine distances  and storing best dist with name
        for rep in reps:
            emb = rep['embedding']
            for person, embs in embeddings.items():
                for known_emb in embs:
                    dist = cosine(emb, known_emb)
                    if dist < best_dist:
                        second_best = best_dist
                        best_dist = dist
                        best_person = person
                    elif dist < second_best:
                        second_best = dist


        # adding in pred_history to keep track of common name
        if best_dist < THRESHOLD:
            pred_history.append(best_person)
        else:
            pred_history.append("Unknown")
        print(list(pred_history),best_dist) 
           
        # retriving most common
        most_common = max(list(pred_history), key=pred_history.count)
        if(len(list(pred_history))<10):
            display_name = "Detecting..."
        elif most_common != "Unknown" and pred_history.count(most_common) >= 5:
            display_name = most_common
        else:
            display_name = "Unknown"


    except Exception as e:
            print("Error:", e)

    cv2.putText(frame, display_name, (50,50),
                cv2.FONT_HERSHEY_SIMPLEX, 1,
                (0,255,0), 2)

    cv2.imshow("face_recognition", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
    
    
cap.release()
cv2.destroyAllWindows()



['Unknown'] 0.661638549599801
['Unknown', 'Unknown'] 0.6632669663926027
['Unknown', 'Unknown', 'Unknown'] 0.6769219019816268
['Unknown', 'Unknown', 'Unknown', 'Unknown'] 0.6716068569360553
['Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown'] 0.6594107609189694
['Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown'] 0.6661617619731717
['Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown'] 0.6299011217584858
['Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown'] 0.642002322354116
['Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown'] 0.6111498561392019
['Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown'] 0.6591163869936674
['Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown'] 0.6499915674637586
['Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 'Unknown', 

In [6]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))
import torch
print(torch.cuda.is_available())



[]
False


In [1]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


True
NVIDIA GeForce RTX 3050 Laptop GPU


In [2]:
import numpy as np
from deepface import DeepFace

print("Warming up models on GPU...")
dummy = np.zeros((224, 224, 3), dtype=np.uint8)

DeepFace.represent(
    dummy,
    model_name="ArcFace",
    detector_backend="yolov8n",
    enforce_detection=False
)

print("Models loaded on GPU ✅")



Warming up models on GPU...
Models loaded on GPU ✅
